In [13]:
import panel as pn
import pandas as pd
import plotly.express as px
import numpy as np
from shapely.geometry import Point
import random

# 페이지 전체 너비 설정
PAGE_WIDTH = 1600
CONTENT_WIDTH = 1600

# CSS 스타일 정의
css = """
body {
    margin: 0;
    padding: 0;
    width: 100%;
    height:  100%;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background-color: #f5f5f5;
}
.dashboard-container {
    width: 100%;
    max-width: """+str(PAGE_WIDTH)+"""px;
    margin: 0 auto;
    padding: 20px;
    box-sizing: border-box;
}
.dashboard-header {
    background-color: #ffffff;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    margin-bottom: 20px;
}
.dashboard-content {
    background-color: #ffffff;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}
.map-panel, .chart-panel, .info-panel {
    background-color: #f9f9f9;
    border-radius: 8px;
    padding: 15px;
    margin-bottom: 15px;
}
.info-panel {
    background-color: #e9f7fe;
    border-left: 4px solid #4a90e2;
}
"""

# Panel 확장 설정 - Plotly를 지원하도록 명시적으로 설정
pn.extension('plotly', raw_css=[css])

# 샘플 공간 데이터 생성 함수
def generate_spatial_data():
    cities = {
        '서울': (126.9780, 37.5665),
        '부산': (129.0756, 35.1796),
        '인천': (126.7052, 37.4563),
        '대구': (128.6014, 35.8714),
        '광주': (126.8526, 35.1595),
        '대전': (127.3845, 36.3504),
        '울산': (129.3114, 35.5384),
        '세종': (127.2890, 36.4800),
        '수원': (127.0286, 37.2636),
        '청주': (127.4895, 36.6424),
        '전주': (127.1480, 35.8242),
        '포항': (129.3650, 36.0190),
        '제주': (126.5219, 33.5097)
    }

    populations = {'서울': 9700000, '부산': 3400000, '인천': 2900000, '대구': 2400000, '광주': 1500000, '대전': 1500000,
                   '울산': 1100000, '세종': 350000, '수원': 1200000, '청주': 850000, '전주': 650000, '포항': 500000, '제주': 670000}

    visitors = {'서울': 12500000, '부산': 8700000, '인천': 4300000, '대구': 3100000, '광주': 2200000, '대전': 2000000,
                '울산': 1500000, '세종': 800000, '수원': 3500000, '청주': 1200000, '전주': 2800000, '포항': 1700000, '제주': 15000000}

    temperatures = {'서울': 12.5, '부산': 14.7, '인천': 12.1, '대구': 14.1, '광주': 13.8, '대전': 13.1, '울산': 14.2,
                    '세종': 12.9, '수원': 12.3, '청주': 12.8, '전주': 13.3, '포항': 14.3, '제주': 15.8}

    months = ['1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월']
    monthly_visitors = {}

    for city, annual in visitors.items():
        if city == '제주':
            base = [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.13, 0.15, 0.09, 0.08, 0.06, 0.04]
        elif city in ['서울','부산']:
            base = [0.07, 0.07, 0.08, 0.09, 0.09, 0.09, 0.1, 0.1, 0.09, 0.08, 0.07, 0.07]
        else:
            base = [0.06, 0.06, 0.07, 0.09, 0.1, 0.1, 0.11, 0.12, 0.09, 0.08, 0.06, 0.06]
        base = [d * random.uniform(0.9, 1.1) for d in base]
        s = sum(base)
        monthly_visitors[city] = [int(annual * d / s) for d in base]

    # GeoPandas 대신 일반 DataFrame 사용
    df = pd.DataFrame([{
        'city': c, 'longitude': lon, 'latitude': lat,
        'population': populations[c],
        'annual_visitors': visitors[c],
        'temperature': temperatures[c]
    } for c, (lon, lat) in cities.items()])

    monthly_df = pd.DataFrame([
        {'city': c, 'month': m, 'visitors': monthly_visitors[c][i]}
        for c in cities for i, m in enumerate(months)
    ])

    return df, monthly_df

df, monthly_df = generate_spatial_data()
df['visitor_population_ratio'] = df['annual_visitors'] / df['population']

def create_map_plotly(df):
    sizes = np.log1p(df['population']) / np.log1p(df['population'].max()) * 30 + 5
    fig = px.scatter_mapbox(
        df, lat='latitude', lon='longitude', color='visitor_population_ratio',
        size=sizes, color_continuous_scale='plasma', hover_name='city',
        hover_data={'latitude': False, 'longitude': False, 'visitor_population_ratio': ':.2f',
                    'population': True, 'annual_visitors': True, 'temperature': True},
        labels={'visitor_population_ratio': '관광 의존도'}, zoom=6,
        center={"lat": 36.0, "lon": 128.0}, height=600, width=800,
        mapbox_style='open-street-map')
    fig.update_layout(title='대한민국 주요 도시 관광 지표', margin={"r":0,"t":40,"l":0,"b":0},
                      coloraxis_colorbar=dict(title="관광 의존도", 
                                            tickvals=[df['visitor_population_ratio'].min(), df['visitor_population_ratio'].max()], 
                                            ticktext=["낮음", "높음"]))
    return fig

def create_visitor_bar_chart(df):
    fig = px.bar(df.sort_values('annual_visitors', ascending=False), x='city', y='annual_visitors', color='city',
                 title='도시별 연간 방문자 수', height=400, width=700)
    fig.update_layout(xaxis={'categoryorder': 'total descending', 'tickangle': 45}, yaxis_title='연간 방문자 수')
    return fig

def create_monthly_line_chart(monthly_df, city):
    data = monthly_df[monthly_df['city'] == city]
    fig = px.line(data, x='month', y='visitors', title=f'{city} 월별 방문자 추이', height=350, width=700)
    fig.update_traces(line=dict(width=3, color='#1f77b4'))
    return fig

def create_scatter_plot(df):
    fig = px.scatter(df, x='temperature', y='visitor_population_ratio', color='city',
                     title='평균 기온과 관광 의존도 관계', height=400, width=700,
                     hover_data=['population', 'annual_visitors'])
    return fig

main_map = create_map_plotly(df)
visitor_bar = create_visitor_bar_chart(df)
scatter = create_scatter_plot(df)
selected_city = '서울'

monthly_panel = pn.Column(css_classes=['chart-panel'])
city_info_panel = pn.Column(css_classes=['info-panel'])

# 초기 값 설정
monthly_panel.append(pn.pane.Plotly(create_monthly_line_chart(monthly_df, selected_city)))
city_info_panel.append(pn.pane.Markdown(f"""
## {selected_city} 도시 정보
- **인구**: {df.loc[df['city']==selected_city, 'population'].values[0]:,}명
- **연간 방문자 수**: {df.loc[df['city']==selected_city, 'annual_visitors'].values[0]:,}명
- **평균 기온**: {df.loc[df['city']==selected_city, 'temperature'].values[0]}°C
- **관광 의존도**: {df.loc[df['city']==selected_city, 'visitor_population_ratio'].values[0]:.2f}
"""))

city_selector = pn.widgets.Select(name='도시 선택', options=list(df['city']), value=selected_city)

def update_monthly_chart(event):
    city = event.new
    monthly_panel.clear()
    monthly_panel.append(pn.pane.Plotly(create_monthly_line_chart(monthly_df, city)))
    info = df[df['city'] == city].iloc[0]
    city_info_panel.clear()
    city_info_panel.append(pn.pane.Markdown(f"""
## {city} 도시 정보
- **인구**: {info['population']:,}명
- **연간 방문자 수**: {info['annual_visitors']:,}명
- **평균 기온**: {info['temperature']}°C
- **관광 의존도**: {info['visitor_population_ratio']:.2f}
"""))

city_selector.param.watch(update_monthly_chart, 'value')

dashboard_info = pn.pane.Markdown("""
# 대한민국 주요 도시 관광 데이터 대시보드
도시별 관광 통계를 시각화하고 비교할 수 있는 대시보드입니다.
""")

map_info = pn.pane.Markdown("""
## 공간 데이터 시각화 이해하기
- 위치: 도시의 위도, 경도
- 색상: 관광 의존도
- 크기: 인구 규모
""", css_classes=['info-panel'])

page1 = pn.Column(
    dashboard_info,
    pn.Row(
        pn.Column(map_info, pn.pane.Plotly(main_map), css_classes=['map-panel'], width=800),
        pn.Spacer(width=40),
        pn.Column(city_selector, city_info_panel, monthly_panel, width=700)
    ),
    css_classes=['dashboard-content'], width=CONTENT_WIDTH
)

page2 = pn.Column(
    pn.pane.Markdown("# 도시별 관광 데이터 분석"),
    pn.Row(
        pn.Column(pn.pane.Markdown("## 도시별 연간 방문자 수"), pn.pane.Plotly(visitor_bar), css_classes=['chart-panel']),
        pn.Column(pn.pane.Markdown("## 평균 기온과 관광 의존도 관계"), pn.pane.Plotly(scatter), css_classes=['chart-panel'])
    ),
    pn.pane.Markdown("""
## 관광 데이터 분석 인사이트
1. 제주도는 관광 의존도가 매우 높습니다.
2. 대도시는 방문자는 많지만 의존도는 낮습니다.
3. 기온이 높은 도시일수록 관광 의존도가 높을 수 있습니다.
4. 여름철 방문자 증가 경향이 있습니다.
""", css_classes=['info-panel']),
    css_classes=['dashboard-content'], width=CONTENT_WIDTH
)

tabs = pn.Tabs(("지도 및 도시 정보", page1), ("관광 통계 분석", page2), width=CONTENT_WIDTH)

dashboard = pn.Column(pn.pane.Markdown("# 대한민국 관광 데이터 시각화 대시보드", align='center'), tabs,
                      css_classes=['dashboard-container'], width=PAGE_WIDTH)

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
dashboard.save('korea_tourism_dashboard.html', embed=True, resources='cdn')